# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model
- Compare and contrast model validation strategies

## Let's Get Started

We included the code to pre-process the Ames Housing dataset below. This is done for the sake of expediency, although it may result in data leakage and therefore overly optimistic model metrics.

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

## Train-Test Split

Perform a train-test split with a test set of 20% and a random state of 4.

In [60]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [61]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

### Fit a Model

Fit a linear regression model on the training set

In [62]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

In [63]:
# Instantiate and fit a linear regression model
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

### Calculate MSE

Calculate the mean squared error on the test set

In [64]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error, make_scorer

In [65]:
# Calculate MSE on test set
mean_squared_error(y_test, linreg.predict(X_test))

0.17595331097301375

## Cross-Validation using Scikit-Learn

Now let's compare that single test MSE to a cross-validated test MSE.

In [66]:
# Import cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score

In [67]:
# Find MSE scores for a 5-fold cross-validation
mse_scores = cross_val_score(linreg, X_test, y_test, cv=5, scoring=make_scorer(mean_squared_error))

In [68]:
# Get the average MSE score
mse_scores.mean()

0.2149567958999672

Compare and contrast the results. What is the difference between the train-test split and cross-validation results? Do you "trust" one more than the other?

In [69]:
# Your answer here
# the cross validation i trust more due to it using more of our original sample

## Level Up: Let's Build It from Scratch!

### Create a Cross-Validation Function

Write a function `kfolds(data, k)` that splits a dataset into `k` evenly sized pieces. If the full dataset is not divisible by `k`, make the first few folds one larger then later ones.

For example, if you had this dataset:

In [70]:
example_data = pd.DataFrame({
    "color": ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]
})
example_data

,color
0,red
1,orange
2,yellow
3,green
4,blue
5,indigo
6,violet


`kfolds(example_data, 3)` should return:

* a dataframe with `red`, `orange`, `yellow`
* a dataframe with `green`, `blue`
* a dataframe with `indigo`, `violet`

Because the example dataframe has 7 records, which is not evenly divisible by 3, so the "leftover" 1 record extends the length of the first dataframe.

In [71]:
def kfolds(data, k):
    folds = []
    
    # Your code here
    size = data.shape[0] // k
    start = 0
    data = data.iloc[::-1]
    for i in range(k):
        if i < k-1:
            folds.append(data.iloc[start:start+size])
            start += size
        else:
            folds.append(data.iloc[start:])
    
    return folds

In [72]:
results = kfolds(example_data, 3)
for result in results:
    print(result, "\n")

    color
6  violet
5  indigo 

   color
4   blue
3  green 

    color
2  yellow
1  orange
0     red 



### Apply Your Function to the Ames Housing Data

Get folds for both `X` and `y`.

In [73]:
# Apply kfolds() to ames_data with 5 folds
ames_folds = kfolds(preprocessed, 5)

### Perform a Linear Regression for Each Fold and Calculate the Test Error

Remember that for each fold you will need to concatenate all but one of the folds to represent the training data, while the one remaining fold represents the test data.

In [74]:
# Replace None with appropriate code
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X, y = ames_folds[n].iloc[:, :-1], ames_folds[n].iloc[:, -1:]
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    # Fit a linear regression model
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    
    # Evaluate test errors
    test_errs.append(mean_squared_error(y_test, linreg.predict(X_test)))

print(test_errs)

[0.003344165695841907, 0.007835842300784398, 0.010899075616970955, 0.003287331682201862, 0.006353390570905166]


If your code was written correctly, these should be the same errors as scikit-learn produced with `cross_val_score` (within rounding error). Test this out below:

In [75]:
# Compare your results with sklearn results
np.mean(test_errs), mse_scores.mean()

(0.006343961173340858, 0.2149567958999672)

This was a bit of work! Hopefully you have a clearer understanding of the underlying logic for cross-validation if you attempted this exercise.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are more robust than train-test split.